In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import scipy.optimize as scopt
import glob
import os

%matplotlib inline

## Setup

In [3]:
def load_json(file):
    '''
    Loads txt file and returns dataframe using the json dict
    '''
    data = {}
    with open(file, encoding='utf-8') as f:
        for line in f:
            d = str(line).replace("'", '"')
            data = json.loads(d)

    return pd.DataFrame(data)

In [4]:
os.chdir(os.getcwd()+'/rss-data')

## Retrieving and loading each trace

In [45]:
files = [file for file in glob.glob("*.txt")]
dfs = [load_json(file) for file in files]

## Preprocessing data & Separating traces by transmitter/MAC

In [38]:
def preprocess(df_list):
    '''
    Computes rolling average for RSS for each trace
    '''
    clean_dfs = []
    for df in df_list:
        df['rss'] = pd.to_numeric(df['rss'])
        df['mean_rss'] = df['rss'].rolling(window=5, win_type='triang').mean()
        df = df.dropna()
        clean_dfs.append(df)
    macs = list(set(df['mac']))
    
    return clean_dfs, macs

## Defining functions for scipy.optimize.curve_fit

In [35]:
def localization(coords, c, g, x0, y0):
    x1, y1 = coords
    return c + g * np.log10(((x1-x0)**2 + (y1-y0)**2)**0.5)

In [110]:
def c_fit(tx):
    popt, pcov = scopt.curve_fit(localization,
                                 (tx.loc_x,tx.loc_y),
                                 tx.rss,
                                 maxfev=250000,
                                 bounds=((-60,2,-20,-20),(30,6,20,20)))
    return popt

In [111]:
def fit(df_list, trace_num=0):
    '''
    Iterates through traces to estimate missing parameters
    '''
    clean_dfs, macs = preprocess(df_list)
    df = clean_dfs[trace_num]
    popts = []
    for mac in macs:
        tx = df[df['mac'] == mac]
        popts.append([str(mac)] + list(c_fit(tx)))
        
    return pd.DataFrame(popts, columns=['mac','c', 'gamma', 'x', 'y'])

## Fitting the data
We iterate through each data trace and get estimates for [C, $\gamma$, x, y]. We then use the known locations of two transmitters to test our model's accuracy. 

```x1, y1 = (6.8, 6.8)
x2, y2 = (-.87, 9.45)```


In [134]:
def go(df_list):
    results = []
    for i in range(len(df_list)):
        results.append(fit(df_list, i))
        
    res_df = pd.concat(results)
    return res_df

In [112]:
fit(dfs, 0)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-58.342421,6.000000,0.274508,7.420461
1,f8:cf:c5:97:e0:9e,-60.000000,2.000000,6.419792,5.503429
2,d8:c4:6a:50:e3:b1,-59.083633,2.072193,6.420000,3.647195
3,ac:9e:17:7d:31:e8,-52.615524,6.000000,6.377579,3.797998


In [91]:
fit(dfs, 1)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-57.079371,6.000000,0.278994,6.728231
1,f8:cf:c5:97:e0:9e,-59.994626,2.000437,2.604670,6.802287
2,d8:c4:6a:50:e3:b1,-59.957415,2.004767,6.420000,2.881014
3,ac:9e:17:7d:31:e8,-53.307031,6.000000,6.445787,3.381887


In [92]:
fit(dfs, 2)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-57.594846,6.000000,1.535323,6.772372
1,f8:cf:c5:97:e0:9e,-60.000000,2.000000,4.567288,6.785382
2,d8:c4:6a:50:e3:b1,-59.742243,2.071617,6.420000,4.673980
3,ac:9e:17:7d:31:e8,-54.457391,6.000000,6.536224,4.226788


In [93]:
fit(dfs, 3)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-55.425762,3.288460,-0.318745,6.823934
1,f8:cf:c5:97:e0:9e,-59.998994,2.000207,6.419835,5.992509
2,d8:c4:6a:50:e3:b1,-59.864824,2.009999,6.419999,3.698717
3,ac:9e:17:7d:31:e8,-52.430636,6.000000,6.430577,4.007756


In [94]:
fit(dfs, 4)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-58.571653,5.666728,-0.123028,6.819489
1,f8:cf:c5:97:e0:9e,-59.997168,2.000202,6.420000,3.449284
2,d8:c4:6a:50:e3:b1,-60.000000,2.000000,4.189498,6.784485
3,ac:9e:17:7d:31:e8,-52.877763,6.000000,6.453153,3.602920


In [95]:
fit(dfs, 5)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-56.999310,6.000000,-0.635528,7.024005
1,f8:cf:c5:97:e0:9e,-60.000000,2.000000,2.848970,6.800106
2,d8:c4:6a:50:e3:b1,-59.999994,2.003661,5.825070,6.773060
3,ac:9e:17:7d:31:e8,-52.400006,6.000000,6.430915,4.230537


In [96]:
fit(dfs, 6)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-55.830226,4.529742,-0.563842,6.822245
1,f8:cf:c5:97:e0:9e,-59.996185,2.000330,6.419935,5.672038
2,d8:c4:6a:50:e3:b1,-59.999805,2.514367,6.420240,3.921428
3,ac:9e:17:7d:31:e8,-54.140527,6.000000,6.512422,3.376388


In [97]:
fit(dfs, 7)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-55.414463,3.008479,-0.272381,6.826635
1,f8:cf:c5:97:e0:9e,-60.000000,2.000000,3.683464,6.790861
2,d8:c4:6a:50:e3:b1,-57.984876,6.000000,6.305743,6.495404
3,ac:9e:17:7d:31:e8,-54.879959,6.000000,6.468580,3.451785


In [98]:
fit(dfs, 8)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-55.455463,2.805273,0.503614,6.820059
1,f8:cf:c5:97:e0:9e,-60.000000,2.000000,2.663603,6.801684
2,d8:c4:6a:50:e3:b1,-60.000000,6.000000,6.362825,5.382261
3,ac:9e:17:7d:31:e8,-53.616646,6.000000,6.377824,3.680786


In [99]:
fit(dfs, 9)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-54.727373,3.444166,-0.168776,6.819429
1,f8:cf:c5:97:e0:9e,-59.998529,2.000126,3.793968,6.792218
2,d8:c4:6a:50:e3:b1,-58.566244,6.000000,6.049033,6.630630
3,ac:9e:17:7d:31:e8,-53.271730,6.000000,6.387399,3.679787


In [100]:
fit(dfs, 10)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-57.675687,6.000000,-0.336317,7.420254
1,f8:cf:c5:97:e0:9e,-59.973926,2.001981,2.686420,6.801596
2,d8:c4:6a:50:e3:b1,-58.776993,2.311456,6.420000,4.295704
3,ac:9e:17:7d:31:e8,-53.569126,6.000000,6.444649,4.163206


In [101]:
fit(dfs, 11)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-54.537816,2.004726,2.678951,6.801652
1,f8:cf:c5:97:e0:9e,-59.992521,2.000351,2.861732,6.800112
2,d8:c4:6a:50:e3:b1,-60.000000,5.484009,5.699385,6.697405
3,ac:9e:17:7d:31:e8,-52.632837,6.000000,6.434146,3.855694


In [102]:
fit(dfs, 12)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-58.025207,6.000000,0.217810,7.561367
1,f8:cf:c5:97:e0:9e,-59.999857,2.000146,5.888945,6.773726
2,d8:c4:6a:50:e3:b1,-59.775239,2.009028,6.029825,6.773302
3,ac:9e:17:7d:31:e8,-54.679865,6.000000,6.360274,3.922059


In [103]:
fit(dfs, 13)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-53.673726,2.914984,2.114692,6.833970
1,f8:cf:c5:97:e0:9e,-60.000000,2.000000,5.521436,6.777968
2,d8:c4:6a:50:e3:b1,-59.687810,2.020391,6.419999,4.111963
3,ac:9e:17:7d:31:e8,-51.632776,6.000000,6.464295,4.083890


In [113]:
fit(dfs, 14)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-56.592995,4.118271,-0.394613,6.826849
1,f8:cf:c5:97:e0:9e,-59.999151,2.000642,6.419999,5.431198
2,d8:c4:6a:50:e3:b1,-59.993725,2.222866,6.420027,4.352866
3,ac:9e:17:7d:31:e8,-50.714596,6.000000,6.441921,4.370968


In [105]:
fit(dfs, 15)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-57.474192,6.000000,-0.071914,7.556910
1,f8:cf:c5:97:e0:9e,-60.000000,2.000000,4.668303,6.784755
2,d8:c4:6a:50:e3:b1,-59.999984,5.270658,6.419484,4.614716
3,ac:9e:17:7d:31:e8,-51.972306,6.000000,6.479205,4.620922


In [106]:
fit(dfs, 16)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-55.136619,2.445077,-0.007557,6.824394
1,f8:cf:c5:97:e0:9e,-60.000000,2.000000,3.183033,6.797363
2,d8:c4:6a:50:e3:b1,-59.897295,6.000000,6.451548,5.330205
3,ac:9e:17:7d:31:e8,-52.680201,6.000000,6.385853,3.583743


In [107]:
fit(dfs, 17)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-55.135475,3.842908,2.038663,6.800560
1,f8:cf:c5:97:e0:9e,-59.995500,2.000454,2.708467,6.801409
2,d8:c4:6a:50:e3:b1,-59.352868,2.152470,6.419996,5.747714
3,ac:9e:17:7d:31:e8,-51.503807,6.000000,6.435824,4.405460


In [133]:
fit(dfs, 18)

,mac,c,gamma,x,y
0,8c:85:90:16:0a:a4,-54.684794,2.550204,1.606267,6.810736
1,f8:cf:c5:97:e0:9e,-59.995726,2.000252,6.420000,6.731507
2,d8:c4:6a:50:e3:b1,-55.906127,2.176566,6.420000,6.078111
3,ac:9e:17:7d:31:e8,-51.299679,6.000000,6.369360,3.771573


In [135]:
# skip traces 14 and 18, which are computationally expensive
results_df = go(dfs)

### Finding average estimated parameters using all the traces

In [142]:
results_df.groupby(['mac'])['c'].mean()

mac
8c:85:90:16:0a:a4   -56.299977
ac:9e:17:7d:31:e8   -53.098125
d8:c4:6a:50:e3:b1   -59.569954
f8:cf:c5:97:e0:9e   -59.996900
Name: c, dtype: float64

In [138]:
results_df.groupby(['mac'])['gamma'].mean()

mac
8c:85:90:16:0a:a4    4.467679
ac:9e:17:7d:31:e8    6.000000
d8:c4:6a:50:e3:b1    3.407330
f8:cf:c5:97:e0:9e    2.000249
Name: gamma, dtype: float64

In [139]:
results_df.groupby(['mac'])['x'].mean()

mac
8c:85:90:16:0a:a4    0.420263
ac:9e:17:7d:31:e8    6.436747
d8:c4:6a:50:e3:b1    6.158391
f8:cf:c5:97:e0:9e    4.315286
Name: x, dtype: float64

In [140]:
results_df.groupby(['mac'])['y'].mean()

mac
8c:85:90:16:0a:a4    6.980939
ac:9e:17:7d:31:e8    3.886565
d8:c4:6a:50:e3:b1    5.203481
f8:cf:c5:97:e0:9e    6.407455
Name: y, dtype: float64